In [2]:
# 函数式风格
def f1(x, y): return x if x % 2 else x+y
def f2(x, y): return x*y-1
def f2(x: int, y: int) -> int: return x*y-1  # 类型注解，只是注解，不会做类型检查


f2(3, 4)

11

In [3]:
def myapply(ff, data, z=3): return [ff(x, z) for x in data if x > 2]


data = range(10)
myapply(f1, data, 4), myapply(f2, data, 4)

([3, 8, 5, 10, 7, 12, 9], [11, 15, 19, 23, 27, 31, 35])

In [30]:
# 常见的高阶函数
data = [1, 2, 3, 4]
def ff(x): return x**2


print([*map(ff, data)] == [ff(x) for x in data])
[*map(ff, data)]
list(map(ff, data))

True


[1, 4, 9, 16]

In [5]:
def f(x): return x % 2


[*filter(f, data)]

[1, 3]

sort


In [32]:

from functools import reduce
s = ' a=0ttW.ddd,sdf,xbb'
f = [('.', 'W'), ('s', '&'), ('0', '#'), ('bb', '=')]
# 一行代码实现替换
print(*('.', 'W'))
reduce(lambda x, y: x.replace(*y), f, s)
# reduce(lambda x, y: y.replace(*x), f, s)

. W


' a=#ttWWddd,&df,x='

In [7]:
def ff(x, y): return x+y


x = data[0]
for y in data[1:]:
    x = ff(x, y)
x

10

# 装饰器


In [8]:
def f2(x): return x*x
def f3(x): return x+x
def f1(x): return f2 if x else f3


f1(2)(10)
ff1 = f1(0)
ff2 = f1(1)
ff1(6), ff2(6)

(12, 36)

In [34]:
# 返回内置函数
def f1(x):
    def f2(y): return x*y
    return f2


ff = f1(2)
ff(10)

20

## 偏函数


In [10]:
import time
ts = time.sleep
get_time = time.perf_counter  # 获取时间
def pp(x): print('运行时间', int(1000*get_time()-x))
def f1(): print('f1 run'); ts(0.2)
def f2(): print('f2 run'); ts(0.8)
def f3(): print('f3 run'); ts(0.5)

In [11]:
def timeff(f):
    t = get_time()
    f()
    pp(t)


timeff(f1), timeff(f2), timeff(f3)

f1 run
运行时间 77816187
f2 run


运行时间 77817002
f3 run
运行时间 77817502


(None, None, None)

In [12]:
# 装饰器
def timeit(f):  # 这个装饰器只能装饰无参数函数，f，因为wrapper没有参数
    def wrapper():
        t = get_time()
        f()
        pp(t)
    return wrapper

In [35]:
def f4(): print('f4 run'); ts(1)


timeit(f4)()
ff = timeit(f4)
ff()

f4 run
运行时间 80581798
f4 run
运行时间 80582798


In [14]:
# 先调用timeit里面的wrapper，再在wrapper里面调用timeit里面的wrapper，再在wrapper里面调用f5
def timeit2(f):
    def wrapper():
        print("do timeit2")
        f()
    return wrapper


def timeit(f):  # 这个装饰器只能装饰无参数函数，f，因为wrapper没有参数
    def wrapper():
        t = get_time()
        f()
        pp(t)
    return wrapper


# 装饰器的执行顺序，从下往上
print("start")
@timeit2
@timeit
def f5(): print("f5 run"); ts(0.2)

# timeit2(timeit(f5))()
f5()

start
do timeit2
f5 run
运行时间 77818188


In [40]:
def timeit(func):
    def wrapper(*args, **kwargs):
        t = get_time()
        # print(*args, **kwargs)
        print(*args)
        print(kwargs)
        res = func(*args, **kwargs)
        pp(t)
        return res
    return wrapper


@timeit
def test1(xlist):
    ts(0.5)
    return '-'.join(xlist)


@timeit
def test2(x, y, z, w):
    ts(0.1)
    return x*y*z*w


@timeit
def test3():
    ts(0.2)
    return 'ok'


#
test1(['1', '3', '5']), test2(1, 2, z=2, w=4), test3()

['1', '3', '5']
{}
运行时间 80955768
1 2
{'z': 2, 'w': 4}
运行时间 80955876

{}
运行时间 80956079


('1-3-5', 16, 'ok')

In [16]:
import requests
u1 = 'https://www.baidu.com'
requests.get(u1).text[:100]

'<!DOCTYPE html>\r\n<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charse'

In [42]:
def try_counter(cc):  # 装饰器工厂，返回装饰器
    def wrapper1(func):  # 装饰器，接受func作为参数，返回wrapper
        def wrapper(*args, **kwargs):
            c = 0
            while True:
                try:
                    res = func(*args, **kwargs)
                    return res
                except:
                    c += 1
                    print(' crwal ', c)
                    if c < cc:
                        ts(0.2)
                        continue
                    print('bad task')
                    return 0
        return wrapper
    return wrapper1


@try_counter(4)
def getweb(url):
    return requests.get(url).text[:1000]


u2 = 'google.com'
# try_counter(4)(getweb)(u2)
getweb(u1),getweb(u2)

 crwal  1
 crwal  2
 crwal  3
 crwal  4
bad task


('<!DOCTYPE html>\r\n<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charset=utf-8><meta http-equiv=X-UA-Compatible content=IE=Edge><meta content=always name=referrer><link rel=stylesheet type=text/css href=https://ss1.bdstatic.com/5eN1bjq8AAUYm2zgoY3K/r/www/cache/bdorz/baidu.min.css><title>ç\x99¾åº¦ä¸\x80ä¸\x8bï¼\x8cä½\xa0å°±ç\x9f¥é\x81\x93</title></head> <body link=#0000cc> <div id=wrapper> <div id=head> <div class=head_wrapper> <div class=s_form> <div class=s_form_wrapper> <div id=lg> <img hidefocus=true src=//www.baidu.com/img/bd_logo1.png width=270 height=129> </div> <form id=form name=f action=//www.baidu.com/s class=fm> <input type=hidden name=bdorz_come value=1> <input type=hidden name=ie value=utf-8> <input type=hidden name=f value=8> <input type=hidden name=rsv_bp value=1> <input type=hidden name=rsv_idx value=1> <input type=hidden name=tn value=baidu><span class="bg s_ipt_wr"><input id=kw name=wd class=s_ipt value maxlength=255 autocomplete=off a

In [43]:
@try_counter(4)
def getweb(url):
    return requests.get(url).text[:100]


getweb(u1)

'<!DOCTYPE html>\r\n<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charse'

In [19]:
tt = try_counter(2)


@tt
def getweb(url):
    return requests.get(url).text[:100]


getweb(u1)

'<!DOCTYPE html>\r\n<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charse'

In [20]:
# case :生成标记文本
def makeblod(f):
    def inner(s): return '<b>'+f(s)+'</b>'
    return inner


def makeitalic(f):
    def inner(s): return '<i>'+f(s)+'</i>'
    return inner


@makeblod
def makeTxt(s): return s.upper()


makeTxt('hello, lzu XGY')

'<b>HELLO, LZU XGY</b>'

## 函数分析工具箱：调试阶段测试性能，写日志，算资源利用等等


In [21]:
import requests


def getweb(url):
    try:
        return len(requests.get(url).text)
    except:
        return 0


for url in {
    'https://www.baidu.com',
    'https://www.sina.com.cn',
    'https://www.163.com',
}:
    print(url, getweb(url))

https://www.baidu.com 2443
https://www.163.com 235
https://www.sina.com.cn 417572


In [45]:
import requests


def benchmark(ff):
    '''
    执行时间计算
    '''
    def wrapper(*args, **kwargs):
        # wrapper.count += 1
        t = get_time()
        res = ff(*args, **kwargs)
        pp(t)
        return res
    return wrapper


def counter(ff):
    '''
    计算执行次数
    '''
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        print(f"第{wrapper.count}次执行")
        return ff(*args, **kwargs)
    wrapper.count = 0
    return wrapper


def logging(log_file):
    '''
    记录执行日志
    '''
    def wrapper1(ff):
        def wrapper(*args, **kwargs):
            # 把函数名，当前时间，写到logfile
            print(f" write {log_file}.....")
            return ff(*args, **kwargs)
        return wrapper
    return wrapper1


@logging('log.txt')
@benchmark
@counter
def getweb(url):
    return len(requests.get(url).text)


for url in {
    'https://www.baidu.com',
    'https://www.sina.com.cn',
    'https://www.163.com',
}:
    print(url, getweb(url))

 write log.txt.....
第1次执行
运行时间 81494507
https://www.baidu.com 2443
 write log.txt.....
第2次执行
运行时间 81494915
https://www.163.com 235
 write log.txt.....
第3次执行
运行时间 81495773
https://www.sina.com.cn 417368


In [50]:
def ff(): ff.count += 1; print(ff.count)


ff.cc = 0
ff.count = 999
ff.__dict__


1000


In [61]:
ff()

1011


In [24]:
import requests


def getweb(url):
    return len(requests.get(url).text)


def benchmark(ff):
    def wrapper(*args, **kwargs):
        t = get_time()
        res = ff(*args, **kwargs)
        pp(t)
        return res
    return wrapper


def counter(ff):
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        print(f"第{wrapper.count}次执行")
        return ff(*args, **kwargs)
    wrapper.count = 0
    return wrapper


def logging(log_file):
    def wrapper1(ff):
        def wrapper(*args, **kwargs):
            print(f" write {log_file}.....")
            return ff(*args, **kwargs)
        return wrapper
    return wrapper1


# 使用高阶函数替代装饰器
logged_getweb = logging('log.txt')(getweb)
benchmarked_getweb = benchmark(logged_getweb)
counted_getweb = counter(benchmarked_getweb)

for url in {
    'https://www.baidu.com',
    'https://www.sina.com.cn',
    'https://www.163.com',
}:
    print(url, counted_getweb(url))

第1次执行
 write log.txt.....
运行时间 77833321
https://www.baidu.com 15255
第2次执行
 write log.txt.....
运行时间 77834144
https://www.163.com 235
第3次执行
 write log.txt.....
运行时间 77835050
https://www.sina.com.cn 417572
